In [ ]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, GRU, Flatten, TimeDistributed, MaxPool1D, Lambda, Input, Bidirectional, Dropout
from tensorflow.math import reduce_max, reduce_mean
import tensorflow_probability as tfp
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras import Model
from keras.optimizers import SGD
import numpy as np

def format_input(sequence):
  global columns_events
  matrix=np.zeros((1,len(sequence),len(columns_events)))
  n=0
  for i in range(len(sequence)):
    matrix[0,i,:]=(columns_events==sequence[i]).astype('float32')
  return matrix
 
def create_model(model_type):
  assert model_type in [1,2,3,4,5,6,7]
  if model_type==1:
    model = Sequential()
    model.add(SimpleRNN(100, return_sequences=True, input_shape=(100,364), activation='tanh'))
    model.add(TimeDistributed(Dense(1, activation='sigmoid')))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    optimizer = SGD(learning_rate=0.002)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.load_weights('simplernn.hdf5')
  if model_type==2:
    model = Sequential()
    model.add(Input(shape=(100,364)))
    model.add(Bidirectional(SimpleRNN(100, return_sequences=True, activation='tanh')))
    model.add(TimeDistributed(Dense(1, activation='sigmoid')))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    optimizer = SGD(learning_rate=0.002)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.load_weights('bidirectedrnn.hdf5')
  if model_type==3:
    model = Sequential()
    model.add(GRU(100, return_sequences=True, input_shape=(100,364), activation='tanh'))
    model.add(TimeDistributed(Dense(1, activation='sigmoid')))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    optimizer = SGD(learning_rate=0.002)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.load_weights('simplegru.hdf5')
  if model_type==4:
    model = Sequential()
    model.add(Input(shape=(100,364)))
    model.add(Bidirectional(GRU(100, return_sequences=True, activation='tanh')))
    model.add(TimeDistributed(Dense(1, activation='sigmoid')))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    optimizer = SGD(learning_rate=0.002)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.load_weights('bidirectedgru.hdf5')
  if model_type==5:
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(100,364), activation='tanh'))
    model.add(TimeDistributed(Dense(1, activation='sigmoid')))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    optimizer = SGD(learning_rate=0.002)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.load_weights('simplelstm.hdf5')
  if model_type==6:
    model = Sequential()
    model.add(Input(shape=(100,364)))
    model.add(Bidirectional(LSTM(100, return_sequences=True, activation='tanh')))
    model.add(TimeDistributed(Dense(1, activation='sigmoid')))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    optimizer = SGD(learning_rate=0.002)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.load_weights('bidirectedlstm.hdf5')
  if model_type==7:
    input_layer = Input(shape=(100,364))
    gru = GRU(10, return_sequences=True, activation='tanh')(input_layer)
    timedistributed = TimeDistributed(Dense(1, activation='sigmoid'))(gru)
    flatten=Flatten()(timedistributed)
    max_val = Lambda(lambda x: reduce_max(x, axis=1))(flatten)
    mean_val = Lambda(lambda x: reduce_mean(x, axis=1))(flatten)
    difference_layer = Lambda(lambda x: x[0]-x[1])([max_val, mean_val])
    model = Model(inputs=input_layer, outputs=difference_layer)
    optimizer = SGD(learning_rate=0.01)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.load_weights('simplegruownidea.hdf5')
  return model
 
def create_model_reg(model_type):
  assert model_type in [1,2,3,4,5,6,7]
  if model_type==1:
    model = Sequential()
    model.add(GRU(10, return_sequences=True, input_shape=(100,364), activation='tanh',
                  kernel_regularizer=l1(0.1), recurrent_regularizer=l1(0.1)))
    model.add(Dropout(0.1))
    model.add(TimeDistributed(Dense(1, activation='tanh')))
    model.add(Flatten())
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = SGD(learning_rate=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.load_weights('regularizedgrul1dropout.hdf5')
  if model_type==2:
    model = Sequential()
    model.add(GRU(10, return_sequences=True, input_shape=(100,364), activation='tanh',
                  kernel_regularizer=l2(0.1), recurrent_regularizer=l2(0.1)))
    model.add(Dropout(0.1))
    model.add(TimeDistributed(Dense(1, activation='tanh')))
    model.add(Flatten())
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = SGD(learning_rate=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.load_weights('regularizedgrul2dropout.hdf5')
  if model_type==3:
    input_layer = Input(shape=(100,364))
    gru = GRU(10, return_sequences=True, activation='tanh', kernel_regularizer=l2(0.1), recurrent_regularizer=l2(0.1))(input_layer)
    drop = Dropout(0.1)(gru)
    timedistributed = TimeDistributed(Dense(1, activation='sigmoid'))(drop)
    flatten=Flatten()(timedistributed)
    max_val = Lambda(lambda x: reduce_max(x, axis=1))(flatten)
    mean_val = Lambda(lambda x: reduce_mean(x, axis=1))(flatten)
    difference_layer = Lambda(lambda x: x[0]-x[1])([max_val, mean_val])
    model = Model(inputs=input_layer, outputs=difference_layer)
    optimizer = SGD(learning_rate=0.02)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.load_weights('simplegruownideal2dropout.hdf5')
  if model_type==4:
    input_layer = Input(shape=(100,364))
    gru = GRU(10, return_sequences=True, activation='tanh', kernel_regularizer=l2(0.1), recurrent_regularizer=l2(0.1))(input_layer)
    timedistributed = TimeDistributed(Dense(1, activation='sigmoid'))(gru)
    flatten=Flatten()(timedistributed)
    max_val = Lambda(lambda x: reduce_max(x, axis=1))(flatten)
    mean_val = Lambda(lambda x: tfp.stats.percentile(x, 50, axis=1))(flatten)
    difference_layer = Lambda(lambda x: x[0]-x[1])([max_val, mean_val])
    model = Model(inputs=input_layer, outputs=difference_layer)
    optimizer = SGD(learning_rate=0.02)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.load_weights('simplegruownideamedian2.hdf5')
  if model_type==5:
    input_layer = Input(shape=(100,364))
    gru = GRU(10, return_sequences=True, activation='tanh', kernel_regularizer=l2(0.05), recurrent_regularizer=l2(0.05))(input_layer)
    timedistributed = TimeDistributed(Dense(1, activation='sigmoid'))(gru)
    flatten=Flatten()(timedistributed)
    max_val = Lambda(lambda x: reduce_max(x, axis=1))(flatten)
    mean_val = Lambda(lambda x: reduce_mean(x, axis=1))(flatten)
    difference_layer = Lambda(lambda x: x[0]-x[1])([max_val, mean_val])
    model = Model(inputs=input_layer, outputs=difference_layer)
    optimizer = SGD(learning_rate=0.02)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.load_weights('meangrul2.hdf5')
  return model
 
def create_model_semantic(model_type):
  assert model_type in [1,2,3,4,5,6,7]
  if model_type==1:
    input_layer = Input(shape=(100,200))
    gru = GRU(10, return_sequences=True, activation='tanh')(input_layer)
    timedistributed = TimeDistributed(Dense(1, activation='sigmoid'))(gru)
    flatten=Flatten()(timedistributed)
    max_val = Lambda(lambda x: reduce_max(x, axis=1))(flatten)
    mean_val = Lambda(lambda x: reduce_mean(x, axis=1))(flatten)
    difference_layer = Lambda(lambda x: x[0]-x[1])([max_val, mean_val])
    model = Model(inputs=input_layer, outputs=difference_layer)
    model.compile(loss='binary_crossentropy', optimizer='adam')
    model.load_weights('simplegruownideasemantic01197.hdf5')
  return model
 
def get_temporary_as_model(model, sequence, t):
  tasmodel = Model(inputs=model.input, outputs=model.layers[-t].output)
  return tasmodel.predict(sequence)
 
def check_anomaly_occurrences(sequence, anomaly_events):
  occurrences=[]
  for event in anomaly_events:
    if event in sequence:
      occurrences+=list((sequence==event).nonzero()[0])
  return sorted(occurrences)

In [ ]:
input_layer = Input(shape=(50))
 
embedding_layer = Embedding(input_dim=vocabulary_size+1, output_dim=embedding_dim, weights=[weights], trainable=False)
 
embedded_sequences = embedding_layer(input_layer)
 
gru = GRU(10, return_sequences=True, activation='tanh')(embedded_sequences)
 
timedistributed = TimeDistributed(Dense(1, activation='sigmoid'))(gru)
 
flatten=Flatten()(timedistributed)
 
max_val = Lambda(lambda x: reduce_max(x, axis=1))(flatten)
mean_val = Lambda(lambda x: reduce_mean(x, axis=1))(flatten)
 
difference_layer = Lambda(lambda x: x[0]-x[1])([max_val, mean_val])
 
model = Model(inputs=input_layer, outputs=difference_layer)
 